<a href="https://colab.research.google.com/github/Yash-Kamtekar/Special-Topics-Assignment-5/blob/main/continual_learning_using_avalanche.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Continual learning using avalanche library

## SimpleMLP modeling on PermutedMNIST dataset

### Install Avalanche and its dependencies

In [ ]:
!pip install git+https://github.com/ContinualAI/avalanche.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/ContinualAI/avalanche.git to /tmp/pip-req-build-_i6kyg2b
  Running command git clone -q https://github.com/ContinualAI/avalanche.git /tmp/pip-req-build-_i6kyg2b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.9 MB 3.8 MB/s 
     |████████████████████████████████| 529 kB 44.2 MB/s 
     |████████████████████████████████| 427 kB 42.2 MB/s 
     |████████████████████████████████| 532 kB 34.4 MB/s 
     |████████████████████████████████| 168 kB 32.4 MB/s 
     |████████████████████████████████| 182 kB 15.5 MB/s 
     |████████████████████████████████| 63 kB 1.2 MB/s 
     |████████████████████████████████| 166 kB 55.5 MB/s 
     |████████████████████████████████| 166 kB 54.5 MB/s 
     |████████████████████████████████| 162 kB 55.8 MB/s 
     |██

### Create the environment by importing necessary libraries and modules

In [ ]:
import torch

# use CrossEntropyLoss
from torch.nn import CrossEntropyLoss

# use stochastic GD optimizer
from torch.optim import SGD

# import the PermutedMNIST dataset
from avalanche.benchmarks.classic import PermutedMNIST

# import the SimpleMLP dataset
from avalanche.models import SimpleMLP

from avalanche.training import Naive

# utility functions to create plugin metrics
from avalanche.evaluation.metrics import forgetting_metrics, accuracy_metrics,\
    loss_metrics, timing_metrics, cpu_usage_metrics, StreamConfusionMatrix,\
    disk_usage_metrics, gpu_usage_metrics
from avalanche.logging import InteractiveLogger, TextLogger, TensorboardLogger
from avalanche.training.plugins import EvaluationPlugin

### Configure the device settings
Configure the device settings by checking for availability of CUDA GPU. 

Instantiate a simple multi-layer perceptron model.

In [ ]:
# check for CUDA GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# model instantiation
model = SimpleMLP(num_classes = 10)

### Benchmark instantiation
Prepare the benchmark dataset. 

Split the dataset into train and test sets.

In [ ]:
perm_mnist = PermutedMNIST(n_experiences=3)
# split stream into train and test set
train_stream = perm_mnist.train_stream
test_stream = perm_mnist.test_stream

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting /root/.avalanche/data/mnist/MNIST/raw/train-images-idx3-ubyte.gz to /root/.avalanche/data/mnist/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting /root/.avalanche/data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to /root/.avalanche/data/mnist/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting /root/.avalanche/data/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to /root/.avalanche/data/mnist/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting /root/.avalanche/data/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to /root/.avalanche/data/mnist/MNIST/raw



### Define optimizer and loss function for training

In [ ]:
optimizer = SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = CrossEntropyLoss()

In [ ]:
from torch.utils.data import DataLoader

class CustomStrategy():
    """Custom Basic Strategy"""

    def __init__(self, model, optimizer, criterion):
        self.model = model
        self.optimizer = optimizer
        self.criterion = criterion

    def train(self, experience):
        # training loop for each experience (i.e. batch or task).

        train_dataset = experience.dataset
        t = experience.task_label
        train_data_loader = DataLoader(
            train_dataset, num_workers=4, batch_size=128
        )

        for epoch in range(1):
            for mb in train_data_loader:
                # margin
                pass

    def eval(self, experience):
        # custom eval loop for each experience (i.e. batch or task)

        eval_dataset = experience.dataset
        t = experience.task_label
        eval_data_loader = DataLoader(
            eval_dataset, num_workers=4, batch_size=128
        )

        # eval 

### Define the evaluation plugin and loggers
The evaluation plugin manages the metrics computation.
It takes as argument a list of metrics, collectes their results and returns them to the strategy it is attached to.


In [ ]:
# log to Tensorboard
tb_logger = TensorboardLogger()

# log to text file
text_logger = TextLogger(open('log.txt', 'a'))

# print to stdout
interactive_logger = InteractiveLogger()

eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    timing_metrics(epoch=True),
    cpu_usage_metrics(experience=True),
    forgetting_metrics(experience=True, stream=True),
    StreamConfusionMatrix(num_classes=perm_mnist.n_classes, save_image=False),
    disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loggers=[interactive_logger, text_logger, tb_logger]
)

In [ ]:
strategy = Naive(
    model, optimizer, criterion, train_mb_size=32, train_epochs=2,
    eval_mb_size=32, device=device)

### Instantiate Continual learning strategy - Custom Strategy

Devise a training strategy. 

Here, the Custom strategy is implemented.

In [ ]:
stertegy = CustomStrategy(
    model, SGD(model.parameters(), lr=0.001, momentum=0.9),
    CrossEntropyLoss())

### Train and test epochs
Perform training, evaluate the model and log results.

In [ ]:
print('Starting experiment...')
results = []
for experience in perm_mnist.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    # train returns a dictionary which contains all the metric values
    res = strategy.train(experience, num_workers=4)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    # eval also returns a dictionary which contains all the metric values
    results.append(strategy.eval(perm_mnist.test_stream, num_workers=4))

Starting experiment...
Start of experience:  0
Current Classes:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
-- >> Start of training phase << --
0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


100%|██████████| 1875/1875 [01:40<00:00, 18.59it/s]
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.4859
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.8607
100%|██████████| 1875/1875 [01:42<00:00, 18.27it/s]
Epoch 1 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.2603
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9245
-- >> End of training phase << --
Training completed
Computing accuracy on the whole test set
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 313/313 [00:08<00:00, 37.33it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp000 = 0.1842
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.9474
-- Starting eval on experience 1 (Task 0) from test stream --
100%|██████████| 313/313 [00:08<00:00, 36.88it/s]
> Eval on experience 1 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp001 = 2.7480
	Top1

### Printing results
A portion of the output:

In [ ]:
results

[{'Top1_Acc_Epoch/train_phase/train_stream/Task000': 0.9244666666666667,
  'Loss_Epoch/train_phase/train_stream/Task000': 0.2602937007506688,
  'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000': 0.9474,
  'Loss_Exp/eval_phase/test_stream/Task000/Exp000': 0.18417361970245838,
  'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp001': 0.1042,
  'Loss_Exp/eval_phase/test_stream/Task000/Exp001': 2.7479606857299803,
  'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp002': 0.1569,
  'Loss_Exp/eval_phase/test_stream/Task000/Exp002': 2.3939215183258056,
  'Top1_Acc_Stream/eval_phase/test_stream/Task000': 0.4028333333333333,
  'Loss_Stream/eval_phase/test_stream/Task000': 1.775351941252748},
 {'Top1_Acc_Epoch/train_phase/train_stream/Task000': 0.93435,
  'Loss_Epoch/train_phase/train_stream/Task000': 0.22790829817056657,
  'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000': 0.9343,
  'Loss_Exp/eval_phase/test_stream/Task000/Exp000': 0.2188356723740697,
  'Top1_Acc_Exp/eval_phase/test_stream/Ta